In [29]:
# Cell 1: imports and config
import os
import json
import logging
from typing import Any, Dict, List, Optional

import requests
import chromadb
from chromadb.utils import embedding_functions

from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
from py_clob_client.client import ClobClient, ApiCreds
from py_clob_client.constants import POLYGON

# load .env and logger
load_dotenv()
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

# FastMCP instance (not used until Cell 5)
mcp = FastMCP("polymarket")

In [30]:
# Cell 2: CLOB client helper + Chroma init


def _new_clob() -> ClobClient:
    host = os.getenv("CLOB_HOST", "https://clob.polymarket.com")
    key = os.getenv("PK")
    creds = {
        "apiKey": os.getenv("CLOB_API_KEY"),
        "secret": os.getenv("CLOB_SECRET"),
        "passphrase": os.getenv("CLOB_PASS_PHRASE"),
    }
    c = ClobClient(host, key=key, chain_id=POLYGON)
    c.set_api_creds(creds)
    return c


client = _new_clob()
print("CLOB client OK")

CHROMA_DIR = os.getenv("CHROMA_PERSIST_DIR", ".chroma")
chroma_client = chromadb.PersistentClient(path=CHROMA_DIR)
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-mpnet-base-v2"
)
collection = chroma_client.get_or_create_collection(
    name="prediction_markets",
    embedding_function=ef,
)

print("Chroma collection:", collection.name, "contains", collection.count(), "items")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


CLOB client OK


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Chroma collection: prediction_markets contains 47014 items


In [5]:
markets = []
cursor = ""
while True:
    page = client.get_markets(next_cursor=cursor)
    cursor = page.get("next_cursor", "")
    data = page.get("data", [])
    markets.extend(data)
    if not cursor or cursor == "LTE=":
        break
LOGGER.info("Fetched %d markets from CLOB API", len(markets))

INFO:__main__:Fetched 50921 markets from CLOB API


In [21]:
# --- prepare upsert lists ---
ids: List[str] = []
docs: List[str] = []
metas: List[Dict[str, Any]] = []

for m in markets:
    cid = m["condition_id"] or ""
    if not cid:
        continue

    # build a single text blob from every key/value
    parts: List[str] = []
    for k, v in m.items():
        # skip null or empty
        if v is None:
            continue
        if isinstance(v, (str, bool, int, float)) and v != "":
            parts.append(f"{k}: {v}")
        elif isinstance(v, (list, dict)) and v:
            # JSON-dump lists/dicts so they become text
            parts.append(f"{k}: {json.dumps(v, default=str)}")
    text = " ".join(parts)

    # sanitize metadata (only primitives or JSON strings)
    clean_meta: Dict[str, Any] = {}
    for k, v in m.items():
        if isinstance(v, (str, int, float, bool)):
            clean_meta[k] = v
        elif v is None:
            clean_meta[k] = ""
        else:
            clean_meta[k] = json.dumps(v, default=str)

    ids.append(cid)
    docs.append(text)
    metas.append(clean_meta)

In [46]:
chroma_client.heartbeat()

1747227663426525000

In [47]:
for cid, m in zip(ids, metas):
    print(f"{cid}: {m['question']}")

NameError: name 'ids' is not defined

In [48]:
metas

NameError: name 'metas' is not defined

In [32]:
def upsert_in_batches(
    collection,
    ids: List[str],
    documents: List[str],
    metadatas: List[Dict[str, Any]],
    batch_size: int = 5000,
) -> None:
    """
    Upsert in batches of `batch_size` to avoid Chroma's max‐batch‐size limit.
    """
    n = len(ids)
    for i in range(0, n, batch_size):
        slice_ids = ids[i : i + batch_size]
        slice_docs = documents[i : i + batch_size]
        slice_metas = metadatas[i : i + batch_size]
        collection.upsert(
            ids=slice_ids,
            documents=slice_docs,
            metadatas=slice_metas,
        )
        LOGGER.info(f"  Upserted batch {i}–{i+len(slice_ids)} / {n}")

In [33]:
# … inside your index_markets() after you’ve built ids, docs, metas …
upsert_in_batches(collection, ids, docs, metas, batch_size=5000)

Batches: 100%|██████████| 157/157 [01:32<00:00,  1.69it/s]
INFO:__main__:  Upserted batch 0–5000 / 46006
Batches: 100%|██████████| 157/157 [01:37<00:00,  1.61it/s]
INFO:__main__:  Upserted batch 5000–10000 / 46006
Batches: 100%|██████████| 157/157 [02:13<00:00,  1.18it/s]
INFO:__main__:  Upserted batch 10000–15000 / 46006
Batches: 100%|██████████| 157/157 [01:46<00:00,  1.47it/s]
INFO:__main__:  Upserted batch 15000–20000 / 46006
Batches: 100%|██████████| 157/157 [04:55<00:00,  1.88s/it]
INFO:__main__:  Upserted batch 20000–25000 / 46006
Batches: 100%|██████████| 157/157 [05:38<00:00,  2.15s/it]
INFO:__main__:  Upserted batch 25000–30000 / 46006
Batches: 100%|██████████| 157/157 [05:27<00:00,  2.08s/it]
INFO:__main__:  Upserted batch 30000–35000 / 46006
Batches: 100%|██████████| 157/157 [04:15<00:00,  1.63s/it]
INFO:__main__:  Upserted batch 35000–40000 / 46006
Batches: 100%|██████████| 157/157 [04:07<00:00,  1.57s/it]
INFO:__main__:  Upserted batch 40000–45000 / 46006
Batches: 100%|██

In [39]:
collection.count()

46006

In [25]:
# ─── Live‑fetch Helper ─────────────────────────────────────────────────────
def fetch_market_by_id(condition_id: str) -> List[Dict[str, Any]]:
    """
    Fetch a single market live from the CLOB API and shape it
    into the standard MCP output format.
    """
    try:
        m = client.get_market(condition_id=condition_id)
    except Exception as e:
        LOGGER.error("CLOB fetch failed for %s: %s", condition_id, e)
        return []

    outcomes, prices = [], []
    for tok in m.get("tokens", []):
        out = tok.get("outcome")
        pr = tok.get("price")
        if out is None or pr is None:
            continue
        outcomes.append(out)
        try:
            prices.append(float(pr))
        except (TypeError, ValueError):
            prices.append(None)

    return [
        {
            "condition_id": m.get("condition_id"),
            "question_id": m.get("question_id"),
            "slug": m.get("market_slug"),
            "question": m.get("question"),
            "description": m.get("description", ""),
            "outcomes": outcomes,
            "prices": prices,
            "volume": m.get("volume"),
            "active": m.get("active"),
            "closed": m.get("closed"),
            "startDate": m.get("game_start_time") or m.get("start_date_iso"),
            "endDate": m.get("end_date_iso"),
        }
    ]

In [26]:
import regex as re

HEX_RE = re.compile(r"^0x[0-9a-fA-F]{64}$")


def list_all_prediction_markets(
    query: Optional[str] = None, condition_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    - If `condition_id` is set (or if the single positional argument
      *looks* like a 0x-hex ID), fetch just that market live.
    - Else if `query` is set, return up to 10 semantic hits.
    - Otherwise return *all* indexed markets.
    """

    # 1) If they passed a condition_id keyword, or if the first
    #    positional argument is a 0x‐hex string, do a live fetch:
    effective_id = condition_id or (query if HEX_RE.match(query or "") else None)
    if effective_id:
        return fetch_market_by_id(effective_id)

    # 2) Otherwise, do semantic search (or full scan)
    if query:
        resp = collection.query(
            query_texts=[query],
            n_results=10,
            include=["metadatas"],
        )
        metas = resp["metadatas"][0]
        ids = resp["ids"][0]
    else:
        all_ = collection.get(include=["metadatas"])
        metas = all_["metadatas"]
        ids = all_["ids"]

    results: List[Dict[str, Any]] = []
    for cid, m in zip(ids, metas):
        # tokens may be JSON‐stringified, so decode if needed
        raw_tokens = m.get("tokens", [])
        if isinstance(raw_tokens, str):
            try:
                tok_list = json.loads(raw_tokens)
            except json.JSONDecodeError:
                tok_list = []
        else:
            tok_list = raw_tokens

        outcomes, prices = [], []
        for tok in tok_list:
            out = tok.get("outcome")
            pr = tok.get("price")
            if out is None or pr is None:
                continue
            outcomes.append(out)
            try:
                prices.append(float(pr))
            except (TypeError, ValueError):
                prices.append(None)

        results.append(
            {
                "condition_id": cid,
                "question_id": m.get("question_id", ""),
                "slug": m.get("market_slug", ""),
                "question": m.get("question", ""),
                "description": m.get("description", ""),
                "outcomes": outcomes,
                "prices": prices,
                "volume": m.get("volume"),
                "active": m.get("active"),
                "closed": m.get("closed"),
                "startDate": m.get("game_start_time") or m.get("end_date_iso"),
                "endDate": m.get("end_date_iso"),
            }
        )

    return results

In [34]:
list_all_prediction_markets("trump")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.85it/s]


[{'condition_id': '0xcd1b6b71a1964f15e2c14809594cbfa0d576270e8ef94c8c24913121097e09e5',
  'question_id': '0xfa5d43be36255764600be8bf2a29f288e6c8963c121df8400ddfe6bcbc744700',
  'slug': 'will-donald-trump-win-the-popular-vote-in-the-2024-presidential-election',
  'question': 'Will Donald Trump win the popular vote in the 2024 Presidential Election?',
  'description': 'This market will resolve to “Yes” if Donald Trump is the candidate who receives the most votes in the 2024 U.S. presidential election, otherwise this market will resolve to “No.”\n\nIn case of a tie, this market will resolve in favor of the candidate whose last name comes first in alphabetical order.\n\nThe resolution source will be an overwhelming consensus of credible reporting as to who is the winner of the popular vote.',
  'outcomes': ['Yes', 'No'],
  'prices': [1.0, 0.0],
  'volume': None,
  'active': True,
  'closed': True,
  'startDate': '2024-11-05T00:00:00Z',
  'endDate': '2024-11-05T00:00:00Z'},
 {'condition_id'

In [35]:
list_all_prediction_markets("Will Trump be re-elected in 2024?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 17.99it/s]


[{'condition_id': '0x4372614786010b9b7fb52cffa0bdc1359ea81c7cbc75c5bbc619c68326692b37',
  'question_id': '0x3840893eedd0e6cf2e2f3449c79153eb026360b3aa3a59fec13373bbb5fb9804',
  'slug': 'will-donald-trump-jr-win-the-2024-republican-presidential-nomination',
  'question': 'Will Donald Trump Jr. win the 2024 Republican Presidential Nomination?',
  'description': 'This market will resolve to “Yes” if Donald Trump Jr. wins the 2024 nomination of the Republican Party for U.S. president. Otherwise, this market will resolve to “No”. \n\nThe resolution source for this market will be a consensus of official GOP sources, including https://www.gop.com. Any replacement of the republican nominee before election day will not change the resolution of the market.',
  'outcomes': ['Yes', 'No'],
  'prices': [0.0, 1.0],
  'volume': None,
  'active': True,
  'closed': True,
  'startDate': '2024-07-13T00:00:00Z',
  'endDate': '2024-07-13T00:00:00Z'},
 {'condition_id': '0xe2e4442197df901aa5b7c022330eb409a2c1

In [36]:
fetch_market_by_id("0x4cf761852c566a33509bcc5c483257d3053be5aae33ba7462a941ee84debc100")

[{'condition_id': '0x4cf761852c566a33509bcc5c483257d3053be5aae33ba7462a941ee84debc100',
  'question_id': '0xe3b1cbb13c8a0a36809395c274d5f480871542d8bc4db4d6f50f32389a91de10',
  'slug': 'nhl-sea-ari-2023-04-03',
  'question': 'NHL: Seattle Kraken vs. Arizona Coyotes 2023-04-03',
  'description': 'In the upcoming NHL game, scheduled for April 3 at 10:00 PM ET:\n\nIf the Seattle Kraken win, the market will resolve to “Kraken”.\n\nIf the Arizona Coyotes win, the market will resolve to “Coyotes”.\n\n If the game is not completed by April 21, 2023 (11:59:59 PM ET), the market will resolve 50-50.',
  'outcomes': ['Kraken', 'Coyotes'],
  'prices': [1.0, 0.0],
  'volume': None,
  'active': True,
  'closed': True,
  'startDate': '2023-04-04T02:00:00Z',
  'endDate': '2023-04-03T00:00:00Z'}]

In [78]:
# Cell 5: semantic query, printing description instead of snippet
query = "Will Buffet"
res = collection.query(
    query_texts=[query],
    include=["metadatas"],
)

# Chroma always returns ids, so grab them too
ids = res["ids"][0]
metas = res["metadatas"][0]

for idx, (cid, meta) in enumerate(zip(ids, metas)):
    slug = meta.get("market_slug", "")
    question = meta.get("question", "")
    description = meta.get("description", "")

    # if the description begins with the exact question text, strip it off
    if description.startswith(question):
        # remove the question + any leading punctuation/spaces
        description = description[len(question) :].lstrip(" :–—")

    print(f"\nResult #{idx}:")
    print(f"  condition_id: {cid}")
    print(f"  slug:         {slug}")
    print(f"  question:     {question}")
    print(f"  description:  {description}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 71.89it/s]


Result #0:
  condition_id: 0xf8afd8e73b4cdee0b59be60a9772b2cf896b75c32af3f50d61cfe8e3f1dcbb3e
  slug:         will-buffett-say-charlie-or-munger-during-may-3-shareholders-meeting
  question:     Will Buffett say "Charlie" or "Munger" during May 3 Shareholders Meeting?
  description:  Warren Buffett is scheduled to participate in the Berkshire Hathaway Annual Shareholders Meeting on May 3, 2025, 2:00 PM. You can read more about that here: https://www.berkshirehathaway.com/sharehold.html

This market will resolve to "Yes" if Warren Buffett says the listed term during his appearance at this event. Otherwise, the market will resolve to "No".

Any usage of the term regardless of context will count toward the resolution of this market.

Pluralization/possessive of the term will count toward the resolution of this market, however other forms will NOT count.

Instances where the term is used in a compound word will count regardless of context (e.g. joyful is not a compound word for "joy," how

In [37]:
def list_prediction_market_orderbook(
    condition_id: str,
    snapshot_time: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Exactly the same live CLOB orderbook fetch you had before —
    unchanged shape.
    """
    host = "https://clob.polymarket.com"
    eth_key = os.getenv("PK", "")
    creds = ApiCreds(
        api_key=os.getenv("CLOB_API_KEY", ""),
        api_secret=os.getenv("CLOB_SECRET", ""),
        api_passphrase=os.getenv("CLOB_PASS_PHRASE", ""),
    )
    c = ClobClient(host, key=eth_key, chain_id=137, creds=creds)

    market = c.get_market(condition_id=condition_id)
    question = market.get("question", "")
    out: Dict[str, Any] = {
        "condition_id": condition_id,
        "question": question,
        "orderbooks": {},
    }

    for tok in market.get("tokens", []):
        tid = tok["token_id"]
        outcome = tok["outcome"]
        book = c.get_order_book(token_id=tid)
        bids, asks = book.bids, book.asks

        best_bid = float(bids[0].price) if bids else None
        best_ask = float(asks[0].price) if asks else None
        spread = (
            (best_ask - best_bid)
            if best_bid is not None and best_ask is not None
            else None
        )

        out["orderbooks"][outcome] = {
            "best_bid": best_bid,
            "best_ask": best_ask,
            "spread": spread,
            "bids": [{"price": lvl.price, "size": lvl.size} for lvl in bids],
            "asks": [{"price": lvl.price, "size": lvl.size} for lvl in asks],
        }

    return out

In [38]:
list_prediction_market_orderbook(
    "0xd0393d0b0f8dca4d5e64d831fd41491d4657f07ff90b69be1f614c87aeb52580"
)

PolyApiException: PolyApiException[status_code=404, error_message={'error': 'No orderbook exists for the requested token id'}]

In [6]:
def _fetch_interval(
    condition_id: str,
    interval: str,
    fidelity: int,
    start_ts: Optional[int],
    end_ts: Optional[int],
) -> Dict[str, Any]:
    CLOB = "https://clob.polymarket.com"
    m = requests.get(f"{CLOB}/markets/{condition_id}").json()
    raw, all_ts = {}, set()
    for tok in m.get("tokens", []):
        tid, outcome = tok["token_id"], tok["outcome"]
        params = {"market": tid, "fidelity": fidelity}
        if start_ts is not None:
            params["startTs"] = start_ts
        if end_ts is not None:
            params["endTs"] = end_ts
        if start_ts is None and end_ts is None:
            params["interval"] = interval

        h = (
            requests.get(f"{CLOB}/prices-history", params=params)
            .json()
            .get("history", [])
        )
        mapping = {pt["t"]: pt["p"] for pt in h}
        raw[outcome] = mapping
        all_ts.update(mapping.keys())

    ts_sorted = sorted(all_ts)
    series = {o: [raw[o].get(ts) for ts in ts_sorted] for o in raw}
    return {
        "condition_id": condition_id,
        "question": m.get("question", ""),
        "timestamps": ts_sorted,
        "series": series,
        "interval": interval,
    }


def list_prediction_market_graph(
    condition_id: str,
    interval: str = "max",
    fidelity: int = 100,
    start_ts: Optional[int] = None,
    end_ts: Optional[int] = None,
) -> List[Dict[str, Any]]:
    VALID = ["max", "1m", "1w", "1d", "6h", "1h"]
    graphs = {
        iv: _fetch_interval(condition_id, iv, fidelity, start_ts, end_ts)
        for iv in VALID
    }
    data = graphs.get(interval, graphs["1d"])
    return [
        {
            "condition_id": data["condition_id"],
            "question": data["question"],
            "timestamps": data["timestamps"],
            "yes": data["series"].get("Yes", []),
            "no": data["series"].get("No", []),
        }
    ]

In [11]:
list_prediction_market_graph(
    "0x660e04d89d46a4d2a3a372bad6c82fd7a4a37e09e02816df0084dd540062deaf"
)

[{'condition_id': '0x660e04d89d46a4d2a3a372bad6c82fd7a4a37e09e02816df0084dd540062deaf',
  'question': 'Will Buffett say "Inflation" 4+ times during May 3 Shareholders Meeting?',
  'timestamps': [1744638004,
   1744644004,
   1744650005,
   1744656005,
   1744662005,
   1744668004,
   1744674004,
   1744680005,
   1744686004,
   1744692004,
   1744698005,
   1744704004,
   1744710005,
   1744716005,
   1744722005,
   1744728005,
   1744734005,
   1744740005,
   1744746006,
   1744752005,
   1744758005,
   1744764006,
   1744770006,
   1744776005,
   1744782005,
   1744788005,
   1744794006,
   1744800006,
   1744806006,
   1744812006,
   1744818006,
   1744824006,
   1744830005,
   1744836005,
   1744842005,
   1744848006,
   1744854005,
   1744860005,
   1744866005,
   1744872005,
   1744878005,
   1744884005,
   1744890005,
   1744896006,
   1744902006,
   1744908005,
   1744914005,
   1744920005,
   1744926004,
   1744932005,
   1744938005,
   1744944005,
   1744950005,
   1744956005

In [3]:
import talib
from talib import abstract

In [4]:
import yfinance as yf

In [5]:
# Define the ticker symbol
ticker_symbol = "AAPL"

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Fetch historical market data
historical_data = ticker.history(period="1y")  # data for the last year
print("Historical Data:")
print(historical_data)

Historical Data:
                                 Open        High         Low       Close  \
Date                                                                        
2024-05-14 00:00:00-04:00  186.637140  187.423471  185.422818  186.557510   
2024-05-15 00:00:00-04:00  187.035311  189.762547  186.497816  188.836884   
2024-05-16 00:00:00-04:00  189.583386  190.210459  188.777159  188.956314   
2024-05-17 00:00:00-04:00  188.627835  189.921787  188.299370  188.986160   
2024-05-20 00:00:00-04:00  188.448695  191.026635  188.130177  190.150726   
...                               ...         ...         ...         ...   
2025-05-07 00:00:00-04:00  198.909155  199.178806  192.996910  195.992981   
2025-05-08 00:00:00-04:00  197.461064  199.788014  194.425036  197.231369   
2025-05-09 00:00:00-04:00  198.739390  200.277366  197.281295  198.270004   
2025-05-12 00:00:00-04:00  210.970001  211.270004  206.750000  210.789993   
2025-05-13 00:00:00-04:00  210.429993  213.399994  209.0000

In [6]:
# Fetch basic financials
financials = ticker.financials
print("\nFinancials:")
print(financials)


Financials:
                                                        2024-09-30  \
Tax Effect Of Unusual Items                                    0.0   
Tax Rate For Calcs                                           0.241   
Normalized EBITDA                                   134661000000.0   
Net Income From Continuing Operation Net Minori...   93736000000.0   
Reconciled Depreciation                              11445000000.0   
Reconciled Cost Of Revenue                          210352000000.0   
EBITDA                                              134661000000.0   
EBIT                                                123216000000.0   
Net Interest Income                                            NaN   
Interest Expense                                               NaN   
Interest Income                                                NaN   
Normalized Income                                    93736000000.0   
Net Income From Continuing And Discontinued Ope...   93736000000.0   
Total E

In [7]:
# Fetch stock actions like dividends and splits
actions = ticker.actions
print("\nStock Actions:")
print(actions)


Stock Actions:
                           Dividends  Stock Splits
Date                                              
1987-05-11 00:00:00-04:00   0.000536           0.0
1987-06-16 00:00:00-04:00   0.000000           2.0
1987-08-10 00:00:00-04:00   0.000536           0.0
1987-11-17 00:00:00-05:00   0.000714           0.0
1988-02-12 00:00:00-05:00   0.000714           0.0
...                              ...           ...
2024-05-10 00:00:00-04:00   0.250000           0.0
2024-08-12 00:00:00-04:00   0.250000           0.0
2024-11-08 00:00:00-05:00   0.250000           0.0
2025-02-10 00:00:00-05:00   0.250000           0.0
2025-05-12 00:00:00-04:00   0.260000           0.0

[92 rows x 2 columns]


In [40]:
# import numpy as np
# import talib
# import warnings
# from typing import Any, Dict


# def project_time_series(
#     condition_id: str,
#     method_of_prediction: str = "rsi",  # "rsi", "macd", ...
#     lookback_periods: int = 14,
#     prediction_horizon: int = 1,  # Unused for now; placeholder
# ) -> Dict[str, Any]:
#     """
#     Project forward market probability using technical analysis.
#     - condition_id: market ID
#     - method_of_prediction: "rsi" or "macd"
#     - lookback_periods: fallback window size
#     - prediction_horizon: future horizon (not yet implemented)
#     Returns a dict with current & projected probabilities, confidence, indicators, and timestamp.
#     """
#     # 1) Fetch current mid-market probability
#     ob = list_prediction_market_orderbook(condition_id)
#     current_prob = None
#     for outcome, book in ob["orderbooks"].items():
#         if outcome == "Yes":
#             bid, ask = book["best_bid"], book["best_ask"]
#             if bid is not None and ask is not None:
#                 current_prob = (bid + ask) / 2
#                 break
#     if current_prob is None:
#         raise ValueError("Could not determine current market probability")

#     # 2) Fetch historical OHLC series
#     data = _fetch_interval(condition_id, "1h", max(100, lookback_periods), None, None)
#     if not data.get("series"):
#         raise ValueError(f"No price history for market {condition_id}")

#     # Pick "Yes" series if available
#     series_key = "Yes" if "Yes" in data["series"] else next(iter(data["series"]))
#     prices = np.array(
#         [p for p in data["series"][series_key] if p is not None], dtype=float
#     )

#     # Require at least 2 points to compute any TA indicator
#     if prices.size < 2:
#         raise ValueError("Insufficient data: need at least 2 price points")

#     # Determine a safe window for indicators: at most len(prices)-1, at least 2
#     window = min(lookback_periods, prices.size - 1)
#     if window < 2:
#         raise ValueError(f"Adjusted window {window} is too small (min 2)")

#     # Initialize outputs
#     projected_prob = current_prob
#     confidence = 0.5
#     indicators: Dict[str, Any] = {}

#     # 3a) RSI-based projection
#     if method_of_prediction.lower() == "rsi":
#         rsi_values = talib.RSI(prices, timeperiod=window)
#         last_rsi = float(rsi_values[-1]) if not np.isnan(rsi_values[-1]) else 50.0

#         # Adjust probability & confidence
#         if last_rsi > 70:
#             projected_prob *= 0.95
#             confidence = 0.7 if last_rsi > 80 else 0.6
#         elif last_rsi < 30:
#             projected_prob *= 1.05
#             confidence = 0.7 if last_rsi < 20 else 0.6

#         indicators = {
#             "rsi": round(last_rsi, 4),
#             "rsi_trend": (
#                 "bullish"
#                 if last_rsi < 30
#                 else "bearish" if last_rsi > 70 else "neutral"
#             ),
#         }

#     # 3b) MACD-based projection
#     elif method_of_prediction.lower() == "macd":
#         # Ensure MACD periods fit within available data
#         fast, slow, sig = 12, 26, 9
#         if prices.size <= slow:
#             slow = window
#             fast = max(2, window // 2)
#             sig = max(2, window // 3)

#         macd, signal, hist = talib.MACD(
#             prices, fastperiod=fast, slowperiod=slow, signalperiod=sig
#         )
#         last_macd = float(macd[-1]) if not np.isnan(macd[-1]) else 0.0
#         last_signal = float(signal[-1]) if not np.isnan(signal[-1]) else 0.0
#         last_hist = float(hist[-1]) if not np.isnan(hist[-1]) else 0.0

#         if last_macd > last_signal:
#             projected_prob *= 1.05
#             confidence = 0.7 if last_hist > 0 else 0.6
#         else:
#             projected_prob *= 0.95
#             confidence = 0.7 if last_hist < 0 else 0.6

#         indicators = {
#             "macd": round(last_macd, 4),
#             "signal": round(last_signal, 4),
#             "histogram": round(last_hist, 4),
#             "trend": "bullish" if last_macd > last_signal else "bearish",
#         }

#     else:
#         raise NotImplementedError(
#             f"Prediction method '{method_of_prediction}' not implemented"
#         )

#     # 4) Clamp projected probability to [0,1]
#     projected_prob = max(0.0, min(1.0, projected_prob))

#     # 5) Build result
#     return {
#         "condition_id": condition_id,
#         "current_probability": round(current_prob, 4),
#         "projected_probability": round(projected_prob, 4),
#         "confidence": round(confidence, 4),
#         "indicators": indicators,
#         "timestamp": data.get("timestamps", [None])[-1],
#     }

In [41]:
# project_time_series(
#     "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"
# )

In [42]:
ob = list_prediction_market_orderbook(
    "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"
)
current_prob = None
for outcome, book in ob["orderbooks"].items():
    if outcome == "Yes":
        bid, ask = book["best_bid"], book["best_ask"]
        if bid is not None and ask is not None:
            current_prob = (bid + ask) / 2
            break
if current_prob is None:
    raise ValueError("Could not determine current market probability")

In [43]:
bid

0.01

In [44]:
ask

0.99

In [45]:
current_prob

0.5

In [48]:
data = _fetch_interval(
    "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82",
    "max",
    max(100, 14),
    None,
    None,
)

In [49]:
data

{'condition_id': '0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82',
 'question': 'Will Trump remove majority of reciprocal tariffs before 90 day deadline?',
 'timestamps': [1744626005,
  1744632004,
  1744638004,
  1744644004,
  1744650005,
  1744656005,
  1744662005,
  1744668004,
  1744674004,
  1744680005,
  1744686004,
  1744692004,
  1744698005,
  1744704004,
  1744710005,
  1744716005,
  1744722005,
  1744728005,
  1744734005,
  1744740005,
  1744746006,
  1744752005,
  1744758005,
  1744764006,
  1744770006,
  1744776005,
  1744782005,
  1744788005,
  1744794006,
  1744800006,
  1744806006,
  1744812006,
  1744818006,
  1744824006,
  1744830005,
  1744836005,
  1744842005,
  1744848006,
  1744854005,
  1744860005,
  1744866005,
  1744872005,
  1744878005,
  1744884005,
  1744890005,
  1744896006,
  1744902006,
  1744908005,
  1744914005,
  1744920005,
  1744926004,
  1744932005,
  1744938005,
  1744944005,
  1744950005,
  1744956005,
  1744962005,
  1744968006

In [50]:
import numpy as np

series_key = "Yes" if "Yes" in data["series"] else next(iter(data["series"]))
prices = np.array([p for p in data["series"][series_key] if p is not None], dtype=float)

In [51]:
series_key

'Yes'

In [53]:
prices.size

433

In [54]:
window = min(14, prices.size - 1)

In [55]:
projected_prob = current_prob
confidence = 0.5
indicators: Dict[str, Any] = {}

In [56]:
rsi_values = talib.RSI(prices, timeperiod=window)
last_rsi = float(rsi_values[-1]) if not np.isnan(rsi_values[-1]) else 50.0

In [57]:
rsi_values

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       24.55064198, 24.55064198, 24.55064198, 24.55064198, 24.55064198,
       24.55064198, 24.55064198, 24.55064198, 24.55064198, 24.55064198,
       24.55064198, 24.55064198, 24.55064198, 24.55064198, 24.55064198,
       24.55064198, 24.55064198, 24.55064198, 24.55064198, 24.55064198,
        7.765122  ,  7.765122  ,  7.765122  ,  7.765122  ,  7.765122  ,
        7.765122  ,  7.765122  ,  7.765122  ,  4.79781316,  4.79781316,
        4.79781316,  4.79781316,  4.79781316,  4.79781316,  4.79

In [58]:
last_rsi

52.6909442006111

In [59]:
# Adjust probability & confidence
if last_rsi > 70:
    projected_prob *= 0.95
    confidence = 0.7 if last_rsi > 80 else 0.6
elif last_rsi < 30:
    projected_prob *= 1.05
    confidence = 0.7 if last_rsi < 20 else 0.6

indicators = {
    "rsi": round(last_rsi, 4),
    "rsi_trend": (
        "bullish" if last_rsi < 30 else "bearish" if last_rsi > 70 else "neutral"
    ),
}

In [60]:
indicators

{'rsi': 52.6909, 'rsi_trend': 'neutral'}

In [ ]:
fast, slow, sig = 12, 26, 9
if prices.size <= slow:
    slow = window
    fast = max(2, window // 2)
    sig  = max(2, window // 3)

macd, signal, hist = talib.MACD(prices,
                                fastperiod=fast,
                                slowperiod=slow,
                                signalperiod=sig)
last_macd   = float(macd[-1])   if not np.isnan(macd[-1])   else 0.0
last_signal = float(signal[-1]) if not np.isnan(signal[-1]) else 0.0
last_hist   = float(hist[-1])   if not np.isnan(hist[-1])   else 0.0

if last_macd > last_signal:
    projected_prob *= 1.05
    confidence = 0.7 if last_hist > 0 else 0.6
else:
    projected_prob *= 0.95
    confidence = 0.7 if last_hist < 0 else 0.6

indicators = {
    "macd":      round(last_macd,   4),
    "signal":    round(last_signal, 4),
    "histogram": round(last_hist,   4),
    "trend":     "bullish" if last_macd > last_signal else "bearish"
}

In [49]:
import os
import logging
from typing import Any, Dict, List, Optional
import regex as re
import json

import requests
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
from py_clob_client.client import ClobClient, ApiCreds
from py_clob_client.constants import POLYGON

In [3]:
def list_prediction_market_graph_(
    condition_id: str,
    interval: str = "max",
    fidelity: int = 50,
    start_ts: Optional[int] = None,
    end_ts: Optional[int] = None,
) -> Dict[str, Any]:
    """
    Fetch the historical price trend for the prediction market.
    """
    CLOB = "https://clob.polymarket.com"
    m = requests.get(f"{CLOB}/markets/{condition_id}").json()
    raw, all_ts = {}, set()
    for tok in m.get("tokens", []):
        tid, outcome = tok["token_id"], tok["outcome"]
        params = {"market": tid, "fidelity": fidelity}
        if start_ts is not None:
            params["startTs"] = start_ts
        if end_ts is not None:
            params["endTs"] = end_ts
        if start_ts is None and end_ts is None:
            params["interval"] = interval

        h = (
            requests.get(f"{CLOB}/prices-history", params=params)
            .json()
            .get("history", [])
        )
        mapping = {pt["t"]: pt["p"] for pt in h}
        raw[outcome] = mapping
        all_ts.update(mapping.keys())

    ts_sorted = sorted(all_ts)
    series = {o: [raw[o].get(ts) for ts in ts_sorted] for o in raw}
    return {
        "condition_id": condition_id,
        "question": m.get("question", ""),
        "timestamps": ts_sorted,
        "series": series,
        "interval": interval,
    }

NameError: name 'Optional' is not defined

In [54]:
list_prediction_market_graph_(
    "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"
)

{'condition_id': '0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82',
 'question': 'Will Trump remove majority of reciprocal tariffs before 90 day deadline?',
 'timestamps': [1744644004,
  1744647004,
  1744650005,
  1744653005,
  1744656005,
  1744659005,
  1744662005,
  1744665005,
  1744668004,
  1744671005,
  1744674004,
  1744677004,
  1744680005,
  1744683005,
  1744686004,
  1744689005,
  1744692004,
  1744695005,
  1744698005,
  1744701005,
  1744704004,
  1744707005,
  1744710005,
  1744713005,
  1744716005,
  1744719005,
  1744722005,
  1744725005,
  1744728005,
  1744731005,
  1744734005,
  1744737006,
  1744740005,
  1744743006,
  1744746006,
  1744749005,
  1744752005,
  1744755005,
  1744758005,
  1744761005,
  1744764006,
  1744767006,
  1744770006,
  1744773006,
  1744776005,
  1744779006,
  1744782005,
  1744785005,
  1744788005,
  1744791006,
  1744794006,
  1744797005,
  1744800006,
  1744803006,
  1744806006,
  1744809005,
  1744812006,
  1744815006

In [51]:
# ─── MCP Tool: Simulate Portfolio Impact ──────────────────────────────────
def simulate_portfolio_impact(
    portfolio: List[Dict[str, Any]],
    condition_id: str,
    roi_if_extended: float,
    roi_if_not_extended: float,
) -> Dict[str, Any]:
    """
    Simulate the impact of the tariff pause extension on the portfolio.
    """
    # Fetch prediction market data
    market_data = list_prediction_market_graph(condition_id)

    # Simulate the probability of the tariff extension
    p_extended = market_data["series"].get("Yes", [])[0]
    p_not_extended = 1 - p_extended

    # Calculate ROI scenarios
    total_cost = sum(item["cost"] for item in portfolio)
    total_market = sum(item["market"] for item in portfolio)

    expected_roi = p_extended * roi_if_extended + p_not_extended * roi_if_not_extended
    portfolio_value = total_market * (1 + expected_roi)

    scenario = {
        "extended": {"roi": roi_if_extended, "value": portfolio_value},
        "not_extended": {
            "roi": roi_if_not_extended,
            "value": total_market * (1 + roi_if_not_extended),
        },
    }

    return {
        "p_extended": p_extended,
        "p_not_extended": p_not_extended,
        "expected_roi": expected_roi,
        "scenario": scenario,
        "portfolio_summary": {"total_cost": total_cost, "total_market": total_market},
    }

In [57]:
def simulate_portfolio_impact_(
    portfolio: List[Dict[str, Any]],
    condition_id: str,
    roi_if_extended: float,
    roi_if_not_extended: float,
) -> Dict[str, Any]:
    """
    Simulate the impact of the tariff pause extension on the portfolio.
    """
    # Fetch prediction market data
    market_data = list_prediction_market_graph(condition_id)

    # Simulate the probability of the tariff extension
    p_extended = market_data["series"].get("Yes", [])[0]
    p_not_extended = 1 - p_extended

    # Calculate ROI scenarios
    total_cost = sum(item["cost"] for item in portfolio)
    total_market = sum(item["market"] for item in portfolio)

    expected_roi = p_extended * roi_if_extended + p_not_extended * roi_if_not_extended
    portfolio_value = total_market * (1 + expected_roi)

    scenario = {
        "extended": {"roi": roi_if_extended, "value": portfolio_value},
        "not_extended": {
            "roi": roi_if_not_extended,
            "value": total_market * (1 + roi_if_not_extended),
        },
    }

    return {
        "p_extended": p_extended,
        "p_not_extended": p_not_extended,
        "expected_roi": expected_roi,
        "scenario": scenario,
        "portfolio_summary": {"total_cost": total_cost, "total_market": total_market},
    }

In [55]:
# ─── Example Portfolio Simulation (Use this in the main driver) ───────────────────
portfolio = [
    {"name": "ICICI Prudential Mutual Fund", "cost": 348.67, "market": 438.79},
    {"name": "PPFAS Mutual Fund", "cost": 1000, "market": 2160.45},
    {"name": "SBI Mutual Fund", "cost": 5000, "market": 7908.4},
    {"name": "Mirae Asset Mutual Fund", "cost": 1000, "market": 1767.09},
    {"name": "AXIS Mutual Fund", "cost": 1000, "market": 1468.34},
    {"name": "Invesco Mutual Fund", "cost": 2000, "market": 4025.04},
]

condition_id = "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"
roi_if_extended = 0.03  # 3% ROI if tariff is extended
roi_if_not_extended = -0.05  # -5% ROI if tariff is not extended

In [56]:
simulate_portfolio_impact(
    portfolio,
    condition_id,
    roi_if_extended,
    roi_if_not_extended,
)

{'p_extended': 0.745,
 'p_not_extended': 0.255,
 'expected_roi': 0.009599999999999997,
 'scenario': {'extended': {'roi': 0.03, 'value': 17938.683856000003},
  'not_extended': {'roi': -0.05, 'value': 16879.7045}},
 'portfolio_summary': {'total_cost': 10348.67, 'total_market': 17768.11}}

In [58]:
simulate_portfolio_impact_(
    portfolio,
    condition_id,
    roi_if_extended,
    roi_if_not_extended,
)

{'p_extended': 0.745,
 'p_not_extended': 0.255,
 'expected_roi': 0.009599999999999997,
 'scenario': {'extended': {'roi': 0.03, 'value': 17938.683856000003},
  'not_extended': {'roi': -0.05, 'value': 16879.7045}},
 'portfolio_summary': {'total_cost': 10348.67, 'total_market': 17768.11}}

In [5]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pandas as pd


def project_time_series(
    condition_id: str,
    use_graph_data: bool = True,
    interval: str = "max",
    periods: int = 30,
):
    """
    Project the probabilities and ROI over a given time interval using either market graph or order book data.
    """
    if use_graph_data:
        # Fetch prediction market graph data (probabilities over time)
        graph_data = list_prediction_market_graph(
            condition_id=condition_id, interval=interval
        )
        timestamps = graph_data[0]["timestamps"]
        yes_probs = graph_data[0]["yes"]
        no_probs = graph_data[0]["no"]

        # Convert timestamps to datetime for easier handling (if needed)
        timestamps = pd.to_datetime(timestamps, unit="s")

        # Forecasting probabilities using ARIMA
        model_yes = ARIMA(
            yes_probs, order=(5, 1, 0)
        )  # Example ARIMA model, can be tuned
        model_no = ARIMA(no_probs, order=(5, 1, 0))

        # Fit the model and forecast future probabilities
        model_yes_fit = model_yes.fit()
        model_no_fit = model_no.fit()

        forecast_yes = model_yes_fit.forecast(steps=periods)
        forecast_no = model_no_fit.forecast(steps=periods)

        # Convert forecasts to ROI estimates (you can adjust these values based on your assumptions)
        roi_if_extended = 0.03  # Assumed positive ROI for extended tariffs
        roi_if_not_extended = -0.05  # Assumed negative ROI for non-extension

        # Calculate expected ROI
        forecasted_roi_extended = forecast_yes[-1] * roi_if_extended
        forecasted_roi_not_extended = forecast_no[-1] * roi_if_not_extended

        # Simulate portfolio impact based on the forecasted probabilities
        expected_roi = forecasted_roi_extended * 0.5 + forecasted_roi_not_extended * 0.5

        # Output the scenario analysis (for each time period)
        scenario = {
            "extended": {
                "roi": forecasted_roi_extended,
                "value": 18301.15,
            },  # Sample value
            "not_extended": {
                "roi": forecasted_roi_not_extended,
                "value": 16879.7,
            },  # Sample value
        }

        return {
            "expected_roi": expected_roi,
            "scenario": scenario,
            "forecast_yes": forecast_yes,
            "forecast_no": forecast_no,
        }

    else:
        # Use order book data if needed for a different type of forecast (e.g., based on current bid/ask prices)
        orderbook_data = list_prediction_market_orderbook(condition_id=condition_id)
        # Process orderbook data (use best bid/ask or other heuristics)
        # You can implement forecasting logic based on how these values evolve over time
        return {"message": "Orderbook-based forecast is yet to be implemented"}

In [6]:
def main():
    # Set the condition_id for the relevant prediction market
    condition_id = "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"  # Example ID

    # Call project_time_series with use_graph_data set to True for time series forecasting
    forecast_result = project_time_series(
        condition_id=condition_id,
        use_graph_data=True,  # Set this to False if you want to use orderbook data instead
        interval="1d",  # Interval for time series (can be adjusted)
        periods=30,  # Number of periods to forecast (1d, 7d, 30d, etc.)
    )

    # Print the forecasted results
    print("Forecast Result (Time Series Data):")
    print(f"Expected ROI: {forecast_result['expected_roi']}")
    print("Scenario Analysis:")
    print(f"Extended Tariff ROI: {forecast_result['scenario']['extended']['roi']}")
    print(
        f"Extended Tariff Portfolio Value: {forecast_result['scenario']['extended']['value']}"
    )
    print(
        f"Not Extended Tariff ROI: {forecast_result['scenario']['not_extended']['roi']}"
    )
    print(
        f"Not Extended Tariff Portfolio Value: {forecast_result['scenario']['not_extended']['value']}"
    )

    # If you want to see the forecasted probabilities for 'Yes' and 'No':
    print("Forecasted Probabilities (Yes):", forecast_result["forecast_yes"])
    print("Forecasted Probabilities (No):", forecast_result["forecast_no"])


if __name__ == "__main__":
    main()

NameError: name 'list_prediction_market_graph' is not defined

In [7]:
# Mocking the functions (use the actual implementations in your environment)


def list_prediction_market_graph(condition_id: str, interval: str):
    # You can use actual API calls or data fetching logic here
    return {
        "condition_id": condition_id,
        "timestamps": [1746867006, 1746870006, 1746873006, 1746876006],
        "yes": [0.675, 0.675, 0.65, 0.67],
        "no": [0.325, 0.325, 0.35, 0.33],
    }


def list_prediction_market_orderbook(condition_id: str):
    # Placeholder for orderbook data
    return {
        "condition_id": condition_id,
        "orderbooks": {
            "Yes": {"best_bid": 0.01, "best_ask": 0.99},
            "No": {"best_bid": 0.02, "best_ask": 0.98},
        },
    }


def project_time_series(
    condition_id: str, use_graph_data: bool, interval: str, periods: int
):
    """
    Project the probabilities and ROI over a given time interval using either market graph or order book data.
    """
    if use_graph_data:
        graph_data = list_prediction_market_graph(
            condition_id=condition_id, interval=interval
        )
        timestamps = graph_data["timestamps"]
        yes_probs = graph_data["yes"]
        no_probs = graph_data["no"]
    else:
        orderbook_data = list_prediction_market_orderbook(condition_id)
        # For simplicity, use best bid/ask prices to generate probabilities (you can refine this)
        yes_probs = [orderbook_data["orderbooks"]["Yes"]["best_bid"]]
        no_probs = [orderbook_data["orderbooks"]["No"]["best_bid"]]

    # For simplicity, calculate ROI based on yes/no probabilities (this can be expanded)
    roi_if_extended = 0.03  # Placeholder value
    roi_if_not_extended = -0.05  # Placeholder value
    expected_roi = (
        sum(yes_probs) - sum(no_probs)
    ) * 0.5  # Simple ROI estimation based on probabilities

    # Scenario Analysis based on ROI
    scenario = {
        "extended": {"roi": roi_if_extended, "value": 18301.15},
        "not_extended": {"roi": roi_if_not_extended, "value": 16879.7},
    }

    return {
        "expected_roi": expected_roi,
        "scenario": scenario,
        "forecast_yes": yes_probs,
        "forecast_no": no_probs,
    }


def main():
    # Set the condition_id for the relevant prediction market
    condition_id = "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"  # Example ID

    # Call project_time_series with use_graph_data set to True for time series forecasting
    forecast_result = project_time_series(
        condition_id=condition_id,
        use_graph_data=True,  # Set this to False if you want to use orderbook data instead
        interval="1d",  # Interval for time series (can be adjusted)
        periods=30,  # Number of periods to forecast (1d, 7d, 30d, etc.)
    )

    # Print the forecasted results
    print("Forecast Result (Time Series Data):")
    print(f"Expected ROI: {forecast_result['expected_roi']}")
    print("Scenario Analysis:")
    print(f"Extended Tariff ROI: {forecast_result['scenario']['extended']['roi']}")
    print(
        f"Extended Tariff Portfolio Value: {forecast_result['scenario']['extended']['value']}"
    )
    print(
        f"Not Extended Tariff ROI: {forecast_result['scenario']['not_extended']['roi']}"
    )
    print(
        f"Not Extended Tariff Portfolio Value: {forecast_result['scenario']['not_extended']['value']}"
    )

    # If you want to see the forecasted probabilities for 'Yes' and 'No':
    print("Forecasted Probabilities (Yes):", forecast_result["forecast_yes"])
    print("Forecasted Probabilities (No):", forecast_result["forecast_no"])


if __name__ == "__main__":
    main()

Forecast Result (Time Series Data):
Expected ROI: 0.6699999999999999
Scenario Analysis:
Extended Tariff ROI: 0.03
Extended Tariff Portfolio Value: 18301.15
Not Extended Tariff ROI: -0.05
Not Extended Tariff Portfolio Value: 16879.7
Forecasted Probabilities (Yes): [0.675, 0.675, 0.65, 0.67]
Forecasted Probabilities (No): [0.325, 0.325, 0.35, 0.33]


In [8]:
# Placeholder for the actual market data fetching function (use the actual implementation)
def list_prediction_market_graph(condition_id: str, interval: str):
    # Example: Fetching prediction market data (replace with actual API call or logic)
    return {
        "condition_id": condition_id,
        "timestamps": [1746867006, 1746870006, 1746873006, 1746876006],
        "yes": [0.675, 0.675, 0.65, 0.67],
        "no": [0.325, 0.325, 0.35, 0.33],
    }


def list_prediction_market_orderbook(condition_id: str):
    # Example: Placeholder for orderbook data
    return {
        "condition_id": condition_id,
        "orderbooks": {
            "Yes": {"best_bid": 0.01, "best_ask": 0.99},
            "No": {"best_bid": 0.02, "best_ask": 0.98},
        },
    }


def project_time_series(
    condition_id: str,
    method: str,
    interval: str,
    periods: int,
    roi_if_extended: float,
    roi_if_not_extended: float,
):
    """
    Generalized function to project forward probabilities and calculate ROI for a given market condition.

    :param condition_id: The prediction market's unique identifier
    :param method: The method to use ('graph' or 'orderbook')
    :param interval: The time interval for the forecast (e.g., '1d', '7d', '30d')
    :param periods: The number of periods for the forecast
    :param roi_if_extended: ROI if the extended scenario occurs
    :param roi_if_not_extended: ROI if the non-extended scenario occurs

    :return: Dictionary containing forward probabilities, expected ROI, and value projections
    """

    # Fetch the data based on the selected method
    if method == "graph":
        graph_data = list_prediction_market_graph(condition_id, interval)
        timestamps = graph_data["timestamps"]
        yes_probs = graph_data["yes"]
        no_probs = graph_data["no"]
    elif method == "orderbook":
        orderbook_data = list_prediction_market_orderbook(condition_id)
        yes_probs = [orderbook_data["orderbooks"]["Yes"]["best_bid"]]
        no_probs = [orderbook_data["orderbooks"]["No"]["best_bid"]]
    else:
        raise ValueError("Invalid method provided. Use 'graph' or 'orderbook'.")

    # Calculate ROI for each scenario based on projected probabilities
    expected_roi = (sum(yes_probs) - sum(no_probs)) * 0.5  # Simplified ROI calculation

    # Scenario analysis
    scenario = {
        "extended": {
            "roi": roi_if_extended,
            "value": 18301.15,
        },  # Example ROI and value for extended scenario
        "not_extended": {
            "roi": roi_if_not_extended,
            "value": 16879.7,
        },  # Example ROI and value for not extended
    }

    # Simulate the portfolio's expected ROI based on forward probabilities
    projected_roi = {
        "1d": expected_roi * (1 + 0.01),  # 1-day ROI projection (for simplicity)
        "7d": expected_roi * (1 + 0.05),  # 7-day ROI projection
        "30d": expected_roi * (1 + 0.1),  # 30-day ROI projection
        "90d": expected_roi * (1 + 0.2),  # 90-day ROI projection
        "180d": expected_roi * (1 + 0.3),  # 180-day ROI projection
        "1y": expected_roi * (1 + 0.5),  # 1-year ROI projection
    }

    return {
        "expected_roi": expected_roi,
        "scenario": scenario,
        "projected_roi": projected_roi,
        "forecast_yes": yes_probs,
        "forecast_no": no_probs,
    }


# Example of how to call the function
def main():
    # Set the condition_id for the relevant prediction market
    condition_id = "0xb78982c5836f531ac38f6c5e9da073844864d2e4d7350156226145f3983dfb82"  # Example ID
    method = "graph"  # Can be 'graph' or 'orderbook'
    interval = "1d"  # Interval for time series (1d, 7d, etc.)
    periods = 30  # Number of periods for the forecast

    # ROI values for extended and non-extended tariff scenarios
    roi_if_extended = 0.03
    roi_if_not_extended = -0.05

    # Call project_time_series with the specified parameters
    forecast_result = project_time_series(
        condition_id=condition_id,
        method=method,
        interval=interval,
        periods=periods,
        roi_if_extended=roi_if_extended,
        roi_if_not_extended=roi_if_not_extended,
    )

    # Print the forecasted results
    print("Forecast Result (Time Series Data):")
    print(f"Expected ROI: {forecast_result['expected_roi']}")
    print("Scenario Analysis:")
    print(f"Extended Tariff ROI: {forecast_result['scenario']['extended']['roi']}")
    print(
        f"Extended Tariff Portfolio Value: {forecast_result['scenario']['extended']['value']}"
    )
    print(
        f"Not Extended Tariff ROI: {forecast_result['scenario']['not_extended']['roi']}"
    )
    print(
        f"Not Extended Tariff Portfolio Value: {forecast_result['scenario']['not_extended']['value']}"
    )

    # Print the projected ROI for different time intervals
    print("Projected ROI over different time horizons:")
    for period, roi in forecast_result["projected_roi"].items():
        print(f"{period}: ROI = {roi}")


if __name__ == "__main__":
    main()

Forecast Result (Time Series Data):
Expected ROI: 0.6699999999999999
Scenario Analysis:
Extended Tariff ROI: 0.03
Extended Tariff Portfolio Value: 18301.15
Not Extended Tariff ROI: -0.05
Not Extended Tariff Portfolio Value: 16879.7
Projected ROI over different time horizons:
1d: ROI = 0.6767
7d: ROI = 0.7034999999999999
30d: ROI = 0.737
90d: ROI = 0.8039999999999999
180d: ROI = 0.8709999999999999
1y: ROI = 1.005


In [9]:
def forecast_scenario_probabilities(
    condition_id: str,
    time_horizons_days: List[int] = [1, 7, 30, 90, 180, 365],
) -> List[Dict[str, Any]]:
    """
    Forecasts probability scenarios for different time horizons using ARIMA.
    Returns list of {horizon_days, yes_probability, no_probability} dicts.
    """
    try:
        # Get historical price data using existing tool
        hist_data = list_prediction_market_graph(condition_id=condition_id)
        if (
            not hist_data
            or not isinstance(hist_data, list)
            or "yes" not in hist_data[0]
        ):
            LOGGER.error("Invalid historical data format")
            return []

        data = hist_data[0]
        yes_prices = data["yes"]
        timestamps = data["timestamps"]

        if len(yes_prices) < 10:  # Minimum data points for meaningful forecast
            LOGGER.warning("Insufficient historical data for forecasting")
            return []

        # Create time series dataframe with proper datetime index
        dates = pd.to_datetime(timestamps, unit="s")
        series = pd.Series(yes_prices, index=dates)

        # Handle missing values and resample
        daily_series = series.resample("D").last().ffill()
        if len(daily_series) < 3:
            return []

        # Differencing to make series stationary
        diff_series = daily_series.diff().dropna()

        # Automated ARIMA parameter selection (simplified)
        best_aic = np.inf
        best_order = (1, 1, 1)

        # Simple grid search for ARIMA parameters
        for p in range(2):
            for d in range(2):
                for q in range(2):
                    try:
                        model = ARIMA(daily_series, order=(p, d, q))
                        model_fit = model.fit()
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_order = (p, d, q)
                    except:
                        continue

        # Fit best model
        model = ARIMA(daily_series, order=best_order)
        model_fit = model.fit()

        # Generate forecasts for all horizons
        max_horizon = max(time_horizons_days)
        forecast_steps = min(max_horizon, 365)  # Limit to 1 year
        forecast = model_fit.get_forecast(steps=forecast_steps)

        # Create forecast index map
        forecast_index = {
            i + 1: fc.predicted_mean for i, fc in enumerate(forecast.predicted_mean)
        }

        # Map to requested horizons
        results = []
        for days in sorted(time_horizons_days):
            if days > forecast_steps:
                LOGGER.warning(
                    f"Forecast horizon {days}d exceeds model limit of {forecast_steps}d"
                )
                continue

            yes_prob = max(0, min(1, float(forecast_index[days])))
            results.append(
                {
                    "horizon_days": days,
                    "yes_probability": round(yes_prob, 3),
                    "no_probability": round(1 - yes_prob, 3),
                }
            )

        return results

    except Exception as e:
        LOGGER.error(f"Forecast failed: {str(e)}", exc_info=True)
        return []

NameError: name 'List' is not defined